<a href="https://colab.research.google.com/github/AliJavaheriYekta/Enhanced-MIPGAN/blob/master/CopyMIPGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone "https://github.com/AliJavaheriYekta/MIPGAN1_Pytorch"

Cloning into 'MIPGAN1_Pytorch'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
^C


In [ ]:
%cd MIPGAN1_Pytorch/weights/
! wget https://github.com/lernapparat/lernapparat/releases/download/v2019-02-01/karras2019stylegan-ffhq-1024x1024.for_g_all.pt
%cd ..

In [ ]:
!cp -r resultedImages -d /content/drive/MyDrive/sharedresults

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/MIPGAN1_Pytorch/

/content/drive/MyDrive/MIPGAN1_Pytorch


In [ ]:
!ls

CompareImages.py	 outputs	      source_image
comparePlots.py		 perceptual_model.py  stylegan_layers.py
dnnlib			 __pycache__	      TreeConnect.py
HistogramComparision.py  README.md	      weights
LossCalculation.py	 resultedImages
MIPGAN.ipynb		 save_result


In [ ]:
from __future__ import print_function 
from __future__ import division
!pip install piq
from piq import MultiScaleSSIMLoss, FSIMLoss, MDSILoss, PieAPP

import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision
from torchvision import models, transforms
import math
import time
import os
#import copy
from stylegan_layers import  G_mapping,G_synthesis
from perceptual_model import VGG16_for_Perceptual
from TreeConnect import TreeConnect
from LossCalculation import Losses
from torchvision.utils import save_image
import pandas as pd
from collections import OrderedDict
import glob


print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.10.0+cu111
Torchvision Version:  0.11.1+cu111


In [ ]:
torch.manual_seed(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

def opt2_initializer(device):
  weights = []
  for i in range(3):
    weights.append(torch.tensor(.5, requires_grad=True, device=device))
  return weights

def opt2_remover(prms_to_update2):
  for w in prms_to_update2:
    w.detach().cpu()
 
class MyEnsemble(nn.Module):
    def __init__(self, modelA, modelB, num_ftrs, layers_conf):
        super(MyEnsemble, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        # Remove last linear layer
        self.modelA.fc = TreeConnect(input_dim=num_ftrs, hidden_layers_dim=layers_conf['dim'],  output_dim=512, div_coefs=layers_conf['sparsity']).to('cuda:0')
        self.modelB.fc = TreeConnect(input_dim=num_ftrs, hidden_layers_dim=layers_conf['dim'],  output_dim=512, div_coefs=layers_conf['sparsity']).to('cuda:0')
      
    def weight_init(self):
        self.modelA.fc.weight_init()
        self.modelB.fc.weight_init()

    def forward(self, im1,im2):
        x1 = self.modelA(im1.clone())
        m1 = x1.detach()  # clone to make sure x is not changed by inplace methods
        x2 = self.modelB(im2.clone())
        m2 = x2.detach()
        x = (x1 + x2) / 2.0
        return x, m1, m2
 
def read_image(img_source):
    img = np.array(Image.open(img_source).convert("RGB"))
    return img
 
def image_preprocess(img_source1, img_source2, device):
    img_source1 = transforms.ToTensor()(img_source1).unsqueeze_(0)
    img_source2 = transforms.ToTensor()(img_source2).unsqueeze_(0)
    return img_source1.to(device, dtype = torch.float), img_source2.to(device, dtype = torch.float)
 
def adjust_lr(optimizer, lr):
    for param in optimizer.param_groups:
        param['lr'] = lr
    return optimizer
 
def seperate_losses(loss):
    loss1 = loss[1]
    loss2 = loss[2]
    loss = loss[0]
    return loss, loss1, loss2

def train_model(model, perceptual_net, perceptual_weights, g_synthesis, loss_weights, inputs, optimizer, optimizer2, lr, weighted_loss, num_epochs, device):
    global identity_loss
    identity_loss = 0
    data = []
    data_per = []
    weights = loss_weights
    since = time.time()
    lr1 = lr
    image1 = inputs[0]
    image2 = inputs[1]
    loss = 0
    for epoch in range(num_epochs+1):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)
        # Each epoch has a training and validation phase
        model.train()  # Set model to training mode
        
        # zero the parameter gradients
        optimizer.zero_grad()
        optimizer2.zero_grad()

        morph, out1, out2 = model(image1, image2)
        identity_loss, identity_diff = Losses.identity_loss_calc(out1, out2)
 
        morph = morph.unsqueeze(1).repeat(1, 18, 1)
        synth = g_synthesis(morph)
        
        img_p1=image1.clone() #Perceptual loss
        img_p2=image2.clone()
        upsample2d=torch.nn.Upsample(scale_factor=1024/1024, mode='bilinear')
        img_p1=upsample2d(img_p1)
        img_p2=upsample2d(img_p2)
        img_p = [img_p1, img_p2]
        
        fsim_loss, ms_ssim_loss, perceptual_loss = Losses.caluclate_loss(synth, inputs, perceptual_net, img_p, upsample2d, perceptual_weights, epoch, weighted_loss)

        fsim_loss, fsim_loss1, fsim_loss2  = seperate_losses(fsim_loss)
        ms_ssim_loss, ms_ssim_loss1, ms_ssim_loss2  = seperate_losses(ms_ssim_loss)
        perceptual_loss, perceptual_loss1, perceptual_loss2  = seperate_losses(perceptual_loss)

        loss =  0.0002*perceptual_loss + weights['mssim']*ms_ssim_loss+ weights['fsim']*fsim_loss + weights['id_loss']*identity_loss + weights['id_diff']*identity_diff

        # backward + optimize only if in training phase
        loss.backward()
        optimizer.step()
        if epoch%5==0:
          optimizer2.step()

        detach_losses = []
        for l in [loss, ms_ssim_loss, fsim_loss, perceptual_loss, identity_loss, identity_diff]:
            detach_losses.append(l.detach().cpu().numpy())

        detach_sep_losses = []
        for ls in [ms_ssim_loss1, ms_ssim_loss2, fsim_loss1, fsim_loss2, perceptual_loss1, perceptual_loss2]:
            detach_sep_losses.append(ls.detach().cpu().numpy())

        if epoch%6==0:
            lr = lr*0.95
            optimizer = adjust_lr(optimizer, lr)
 
        if epoch%10==0 or epoch==num_epochs-1:
             print("iter{}: loss -- {}, mssim --{}, fsim --{}, percep_loss --{}, identity_loss --{}, identity_diff --{}".format(epoch,detach_losses[0][0][0],detach_losses[1][0],detach_losses[2][0],detach_losses[3],detach_losses[4][0][0],detach_losses[5][0][0]))
             data.append([epoch, detach_losses[0][0][0],detach_losses[1][0],detach_losses[2][0],
                          detach_losses[3],detach_losses[4][0][0],detach_losses[5][0][0]])
             
             data_per.append([epoch, detach_sep_losses[0][0], detach_sep_losses[1][0],
                              detach_sep_losses[2][0], detach_sep_losses[3][0],
                              detach_sep_losses[4], detach_sep_losses[5]])
             
             synth = (synth - torch.min(synth))/(torch.max(synth)-torch.min(synth))
             save_image(synth.clamp(0,1),"save_result/{}.png".format(epoch))
             
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    return data, data_per
 

def run(perceptual_net, g_synthesis, loss_weights, layers_conf, img1, img2, lr, num_epochs, cnt, weighted_loss='none'):
  device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
  # Create ensemble model
  modelA = models.resnet50(pretrained=True).to(device)
  modelB = models.resnet50(pretrained=True).to(device)
  num_ftrs = modelA.fc.in_features
  # Freeze these models
  for param in modelA.parameters():
      param.requires_grad_(False)
  
  for param in modelB.parameters():
      param.requires_grad_(False)

  model = MyEnsemble(modelA, modelB,num_ftrs, layers_conf).to(device) 

  inputs = [img1.to(device), img2.to(device)]
  prms_to_update = []
  for name, param in model.named_parameters():
      if param.requires_grad == True:
          prms_to_update.append(param)
  prms_to_update2 = opt2_initializer(device)

  data_header = ["iter", "loss", "mssim", "fsim", "perceptual", "identity", "identity diff"]
  data_header2 = ["iter","ms-ssim1", "ms-ssim2", "fsim1", "fsim2", "perceptual1", "perceptual2"]

  optimizer = optim.Adam(prms_to_update, lr=lr, betas=(0.9,0.999))
  optimizer2 = optim.Adagrad(prms_to_update2)
  
  data, data_per = train_model(model, perceptual_net, prms_to_update2, g_synthesis, loss_weights, inputs, optimizer, optimizer2, lr, weighted_loss, num_epochs=num_epochs , device=device)
  data = pd.DataFrame(data, columns=data_header)
  data_per = pd.DataFrame(data_per, columns=data_header2)
  data.to_csv('outputs/output{}.csv'.format(cnt), index=False)
  data_per.to_csv('outputs/output_sep{}.csv'.format(cnt), index=False)

  opt2_remover(prms_to_update2)
  del prms_to_update2
  torch.cuda.empty_cache()


In [ ]:
reference_path = 'resultedImages/' #@param {type: "string"}
source =  13 #@param {type:"number"}
source_folder = 'source_image/{}/'.format(source)
source_images = [source_folder+str(source)+'_1.png', source_folder+str(source)+'_2.png']


In [ ]:
def varInit():
    layers_conf = {'dim': [1024, 512], 'sparsity': [64, 32]}
    # We use pretrained torchvision models here
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'   

    img1 = read_image(source_images[0])
    img2 = read_image(source_images[1])
    img1, img2 = image_preprocess(img1, img2, device)

    weighted_loss = 'none'

    g_synthesis = G_synthesis(resolution=1024)
    g_all = nn.Sequential(OrderedDict([
            ('g_mapping', G_mapping()),
            ('g_synthesis', G_synthesis(resolution=1024))    
            ]))
    
    g_all.load_state_dict(torch.load("weights/karras2019stylegan-ffhq-1024x1024.for_g_all.pt", map_location=device))
    g_all.eval()
    g_all.to(device)
    
    perceptual_net = VGG16_for_Perceptual(n_layers=[2,4,9,16],device=device)
    
    g_synthesis = g_all[1]
    g_synthesis.eval()
    g_synthesis.to(device)
    del g_all
    # Number of epochs to train for 
    num_epochs = 160
    
    # Observe that all parameters are being optimized
    lr = 0.03

    # Train and evaluate
    farthest_setted = False
    farthest_percpt = False
    return perceptual_net, g_synthesis, layers_conf, img1, img2, lr, num_epochs, weighted_loss

if __name__ == "__main__":
  
  cnt = 1
  configs = []

  dir_to_zip = 'save_result/'
  conf_headers = ["number of layers", "weights count", "mssim weight", "fsim weight", "epoch count"]
  #(0.7,0.3)
  #(1.0,0.0),
  weight_tuples = {'train': [(1.0,0.0), (0.75,0.25), (0.6,0.4), (1.0,0.25), (0.75,0.5)],
                   'test': [(0.75,0.25)]}
  
  for wms, wfs in weight_tuples['test']:  
    weights = {'mssim': wms,
              'fsim': wfs,
              'id_loss': 10,
              'id_diff': 1}
    perceptual_net, g_synthesis, layers_conf, img1, img2, lr, num_epochs, weighted_loss = varInit()
    configs.append([layers_conf['dim'], layers_conf['sparsity'], weights['mssim'], weights['fsim'], num_epochs])
    
    run(perceptual_net, g_synthesis, weights, layers_conf, img1, img2, lr, num_epochs, cnt, weighted_loss)  

    output_filename = reference_path + '/file{}.zip'.format(cnt)
    os.system( "zip -r {} {}".format( output_filename , dir_to_zip ) )
    cnt = cnt + 1

  data = pd.DataFrame(configs, columns=conf_headers)
  data.to_csv('outputs/configs.csv', index=False)

In [ ]:
# unzip results
cnt = 0
for zipfile in glob.glob(reference_path + "*.zip"):
  cnt = cnt + 1
  file_to_unzip = zipfile
  output_dir = reference_path + '/conf{}'.format(cnt)
  os.system( "unzip {} -d {}".format( file_to_unzip , output_dir ))
  ref = reference_path + '/conf{}/content/drive/MyDrive/MIPGAN1_Pytorch/save_result'.format(cnt,cnt)
  dest = reference_path + '/conf{}'.format(cnt)
  os.system( "mv {} {}".format( ref , dest ))
  os.system( 'rm -fr {}'.format(reference_path + 'conf{}/content'.format(cnt)))

In [ ]:
!pip install deepface
from deepface import DeepFace

In [ ]:
from CompareImages import CompareImages
cmp = CompareImages()
model = DeepFace.build_model('ArcFace')
cmp.compare([reference_path],[source_folder],model)
os.system( 'rm {}'.format(reference_path + 'ComparisionResults0.csv'))

In [ ]:
from comparePlots import PlotResults

PlotResults().plotResults('resultedImages/')

<Figure size 432x288 with 0 Axes>

In [ ]:
drive.flush_and_unmount()

In [ ]:
# remove folders
for i in range(5):
  path = 'Results{}plots' #@param ['conf{}', 'Results{}plots'] {type:"string"}
  folder = reference_path + path.format(i+1)
  os.system( 'rm -fr {}'.format( folder))

In [ ]:
import os
dir_to_zip = '/content/drive/MyDrive/sharedresults/resultedImages2' #@param {type: "string"}
output_filename = '/content/drive/MyDrive/sharedresults/results2.zip' #@param {type: "string"}
delete_dir_after_download = "No"  #@param ['Yes', 'No']

os.system( "zip {} {}".format( output_filename , dir_to_zip ) )

0